In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
from tensorflow.keras.models import load_model


In [4]:
# Load the trained model, scaler pickel,onehot
model = load_model('my_model.keras')
 
 #load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
     label_encoder_geo = pickle.load(file)
     
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [5]:
#Example input data
input_data = {
    'CreditScore': 600,
    'Age': 40,
    'Geography' : 'France' ,
    'Gender':'Male',
    'Tenure': 3,
    'Balance': 60000,
    'NumofProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Gautam Rahul\Desktop\py\udemy_final\RNN\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Age,Geography,Gender,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,40,France,Male,3,60000,2,1,1,50000


In [8]:
input_df.rename(columns={'NumofProducts': 'NumOfProducts'}, inplace=True)


In [9]:
#Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df



,CreditScore,Age,Geography,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,40,France,1,3,60000,2,1,1,50000


In [10]:
#Concatination one hot encode 
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Age,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,40,1,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
#Scaling the data
# Rename the column to match the one used during fit
# input_df.rename(columns={'NumofProducts': 'NumOfProducts'}, inplace=True)

# # Now transform the data
# input_scaled = scaler.transform(input_df)

# # Check the scaled data
# print(input_scaled)

# Get the correct feature names and order from the fitted scaler
correct_order = scaler.get_feature_names_out()

# Reorder input_df columns to match the order of features used during fit
input_df = input_df[correct_order]

# Now transform the data
input_scaled = scaler.transform(input_df)

# Check the scaled data
print(input_scaled)



[[-0.53598516  0.91324755  0.10479359 -0.69539349 -0.25781119  0.80843615
   0.64920267  0.97481699 -0.87683221  1.00150113 -0.57946723 -0.57638802]]


In [12]:
#predict churn

prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


array([[0.0335183]], dtype=float32)

In [13]:
prediction_prob = prediction[0][0]
 
prediction_prob

np.float32(0.033518296)

In [14]:
if prediction_prob>0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
